In [1]:
import metrics_helpers
import pickle as pk
import gc
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns


In [2]:
def dt_to_int(dt): #datetime to integer
    return dt.astype('int')/(10**9)

In [3]:
#open the facebook data
with open('fb_merged_cleaned_flat.pkl', 'rb') as f:
    fb_search = pk.load(f)

In [5]:
fb_ind = metrics_helpers.indicators(fb_search, fb=True)


In [8]:
#blah = metrics_helpers.clustering(fbtext)

NameError: name 'TfidfVectorizer' is not defined

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.datasets import fetch_20newsgroups
from sklearn.pipeline import Pipeline

from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import NMF, LatentDirichletAllocation

import numpy as np



In [41]:
n_features = 1000
true_k = 10

text_list = list(fb_ind.text_dict.values())
keys = list(fb_ind.text_dict.keys())
#fit and transform data
vectorizer = TfidfVectorizer(max_df=0.5, max_features=n_features,min_df=2, 
                         stop_words='english',use_idf=True)
X = vectorizer.fit_transform(text_list)

km = MiniBatchKMeans(n_clusters=true_k)
km.fit(X)

MiniBatchKMeans(n_clusters=20)

In [42]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

In [19]:
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Cluster 0: com free pm just health information check cake org good
Cluster 1: baby children toddlers kids day toddler child babies pregnancy pregnant
Cluster 2: la et les le en pour di vous du il
Cluster 3: food health baby com safety like new healthy foods help
Cluster 4: know nature offer say vitamin day babyfood house foods healthy
Cluster 5: 100 gate little cow milk ve feeding did years salt
Cluster 6: happy recipe water oil sweet fresh add puree cheese cup
Cluster 7: love time new year just hope baby little amazing good
Cluster 8: organic fruit products baby natural food healthy juice ingredients sugar
Cluster 9: als fda breastfeeding baby health women christmas days nurown care


## Netmums

In [20]:
with open('/Users/sma/Documents/INRAE internship/scrape-git/netmums/allposts_rerun.pkl', 'rb') as f:
    netmums = pk.load(f)

    
nm_ind = metrics_helpers.indicators(netmums, fb=False)

In [21]:
n_features = 1000
true_k = 10

text_list = list(nm_ind.text_dict.values())
keys = list(nm_ind.text_dict.keys())
#fit and transform data
vectorizer = TfidfVectorizer(max_df=0.5, max_features=n_features,min_df=2, 
                         stop_words='english',use_idf=True)
X = vectorizer.fit_transform(text_list)

km = MiniBatchKMeans(n_clusters=true_k)
km.fit(X)

MiniBatchKMeans(n_clusters=10)

In [22]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Cluster 0: children child school year people house kids lol pregnant gp
Cluster 1: milk reflux feed feeding breast breastfeeding formula bottle bf feeds
Cluster 2: eat eating diet weight fruit meat pregnant foods pregnancy healthy
Cluster 3: milk formula cows weaning dairy rice soya bottle babies cow
Cluster 4: bottles water bottle formula powder milk boiled cool hot fridge
Cluster 5: chicken rice weaning fruit veg eat salt cook cheese meat
Cluster 6: water juice sugar drink fruit squash drinks milk teeth drinking
Cluster 7: sleep night cot bed crying waking netmums settle milk sleeping
Cluster 8: skin eczema cream bath rash creams nappy oil dry gp
Cluster 9: sent netmums app mobile iphone sm milk eat pregnant pregnancy


# LDA

## Facebook

In [43]:
def display_topics_full(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(list(fb_ind.text_dict.keys())[doc_index]) #print the corresponding URL
            print(documents[doc_index])
            print("------------------------")

In [44]:
documents = list(fb_ind.text_dict.values())

In [45]:
tf_vectorizer = CountVectorizer(max_df=0.5, min_df=2, max_features=n_features, 
                                stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [46]:
lda = LatentDirichletAllocation(n_components=20, 
                                learning_method="batch").fit(tf)
lda_W = lda.transform(tf)
lda_H = lda.components_

In [47]:
n_top_words = 102
n_top_documents = 10

display_topics_full(lda_H, lda_W, tf_feature_names, documents, 
                    n_top_words, n_top_documents)



Topic 0:
food safety shall authority information act products africa foods industry date raw farmers million packaging poisoning people contamination government 30 waste person climate market 2021 check processing pet technology 000 cases bacteria 10 public safe fine year save consumer control illness product meat farm 2020 november 2016 case means term consumption action including quality conditions drink health 12 video special following process com general months company covid level business human 13 contact production 19 cause international agriculture 11 affected storage possible produce companies 21 nature order result store eat use years risk ensure 20 access number sale best humans dairy come stores
https://www.facebook.com/naeemaliadv/posts/1695054450634905
THE PUNJAB FOOD AUTHORITY ACT 2011

(Act XVI of 2011)

C O N T E N T S



Section Heading



Chapter I

Preliminary

1. Short title, extent and commencement.

2. Definitions.



Chapter II

Establishment of Food Authority



## Netmums

In [48]:
def display_topics_full(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(list(nm_ind.text_dict.keys())[doc_index]) #print the corresponding URL
            #print(documents[doc_index])
            print("------------------------")

In [49]:
documents = list(nm_ind.text_dict.values())

In [50]:
tf_vectorizer = CountVectorizer(max_df=0.5, min_df=2, max_features=n_features, 
                                stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [51]:
lda = LatentDirichletAllocation(n_components=20, 
                                learning_method="batch").fit(tf)
lda_W = lda.transform(tf)
lda_H = lda.components_

In [52]:
n_top_words = 102
n_top_documents = 10

display_topics_full(lda_H, lda_W, tf_feature_names, documents, 
                    n_top_words, n_top_documents)

Topic 0:
yes range product nappy recommend poo brand change aldi tea price products child buy snacks nappies organic ingredients changing definitely favourite important great liked soft family friends currently reason consider loved coffee size normally wipes quality easy toilet pack love tried mum dry colour offer enjoyed prefer nice flavour green daughter taste 10 value kitchen bag decision tomato carrot usually fruit thank natural apple ones parents needs wouldn buying children snack skin perfect absolutely posted especially babies rash toddler maybe clean cheaper lovely felt smell tend shop continue healthy probably normal happy netmums money changed mums bigger choose cost thread haven fact
https://www.netmums.com/coffeehouse/product-tests-sponsored-chat-competitions-251/netmums-product-testing-181/1661207-thread-now-closed-aldi-mamia-organic-food-pouch-product-trial-review-thread.html
------------------------
https://www.netmums.com/coffeehouse/product-tests-sponsored-chat-compet

# 2016 Subst Only

In [56]:
from datetime import datetime
time_cutoff = datetime(2016,1,1, 0, 0, 0, 0)

In [76]:
for key, v in nm_ind.results_dict.items():
    if not v['post_time']:d
        print(key)

KeyError: 'post_time'

In [81]:
'post_time' in [m for i,m in nm_ind.results_dict.items()][0]

True

In [83]:
#generate the dates using command
nm_ind.add_post_time()
#get a subset of keys based on that.
recent_keys = [key for key, value in nm_ind.results_dict.items() if 'post_time' in value and value['post_time'] >= time_cutoff]
# feeds keys into nm text dict to generate subtextdict.

text_list = []
keys = []

for key in recent_keys:
    text_list.append(nm_ind.text_dict[key])
    keys.append(key)

In [ ]:
## Kmean

In [84]:
X = vectorizer.fit_transform(text_list)
km.fit(X)

MiniBatchKMeans(n_clusters=20)

In [85]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Cluster 0: children school child son kids home family parents dad life
Cluster 1: breast breastfeeding feeding milk bottle feed formula fed bottles iphone
Cluster 2: test af pregnancy period pregnant bfp tests positive symptoms cycle
Cluster 3: eczema skin cream bath oil aveeno dry face wash iphone
Cluster 4: chicken sauce pasta add beans rice salt mince eat veg
Cluster 5: smell wash water shower problem clean lol strong house bath
Cluster 6: water bottles bottle formula boiled cool powder milk machine hot
Cluster 7: sugar fruit chocolate aldi buy free cake eat sweets sweet
Cluster 8: iphone sm emoji23 babies wipes bottles lol mouth g930f thank
Cluster 9: weight diet stone lose lost exercise eat eating fat loss
Cluster 10: weaning milk early rice babies ready start veg health hv
Cluster 11: sleep night milk bottle bed feed sleeping babies 30 routine
Cluster 12: house council home clean money kitchen pay cat garden toys
Cluster 13: eat finger foods weaning teeth eating fruit toast veg m

In [86]:
## LDA

In [88]:
def display_topics_full(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(list(nm_ind.text_dict.keys())[doc_index]) #print the corresponding URL
            #print(documents[doc_index])
            print("------------------------")

In [89]:
documents = list(text_list)

In [90]:
tf_vectorizer = CountVectorizer(max_df=0.5, min_df=2, max_features=n_features, 
                                stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [91]:
lda = LatentDirichletAllocation(n_components=20, 
                                learning_method="batch").fit(tf)
lda_W = lda.transform(tf)
lda_H = lda.components_

In [92]:
n_top_words = 102
n_top_documents = 10

display_topics_full(lda_H, lda_W, tf_feature_names, documents, 
                    n_top_words, n_top_documents)

Topic 0:
iphone christmas sm lol emoji170 emoji23 today kids bought g920f love xmas buy 10 set ds 99 dd new ordered look bits stuff hun birthday argos thank oh presents box son amazon ones emoji4 eve price toys great daughter xxx haha looking plus gift order ladies big start ideas bargains 11 lovely boys tree tomorrow seen book 12 sale im g930f santa shopping home boy december couple 16 nice money house idea loves list 20 free 50 delivery online month mum years books picked buying loads thinking girls play ds1 present haven half far school come usually paper happy ds2 boxes shop
https://www.netmums.com/coffeehouse/diets-1061/slimming-world-1069/1787438-sugary-cravings.html
------------------------
https://www.netmums.com/coffeehouse/being-mum-794/children-4-11-years-60/1264578-children-migraines.html
------------------------
https://www.netmums.com/coffeehouse/being-mum-794/toddlers-1-3-years-59/739404-dd-eczema-any-advice-pls.html
------------------------
https://www.netmums.com/coffe